In [ ]:
#csv 파일 저장(컬럼명은 DB에서 나온대로 저장)
from bs4 import BeautifulSoup
import requests
import os
import time
import csv
from datetime import datetime, timedelta
from tqdm import tqdm

base_url = "https://news.naver.com/breakingnews/section/101/262?date="
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36", "Referer": "https://www.nate.com/"
}

output_file = r"C:\Users\cobe2\Documents\SeSAC\PJ\nnews_Global.csv"

def extract_and_save_articles(soup, date, writer):
    li_tg = soup.select('ul.sa_list > li > div.sa_item_inner > div.sa_item_flex > div.sa_thumb > div.sa_thumb_inner > a')
    for i in li_tg:
        crawling_url = i.attrs['href']
        response = requests.get(crawling_url, headers=headers)
        crawling_soup = BeautifulSoup(response.text, 'html.parser')  
        title_tag = crawling_soup.select_one('h2.media_end_head_headline')
        title = title_tag.text if title_tag else "No title Found"       
        date_li_tag = crawling_soup.select_one('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')
        formatted_date = date_li_tag.text.replace("오후", "").replace('.', '_').replace(':', '_').replace(' ', '').replace('오전', '') if date_li_tag else "No Date Found"
        date_mo_tag = crawling_soup.select_one('span.media_end_head_info_datestamp_time._ARTICLE_MODIFY_DATE_TIME')
        modify_date = date_mo_tag.text.replace("오후","").replace('.', '_').replace(':', '_').replace(' ','').replace('오전', '') if date_mo_tag else " "
        source_img_tag = crawling_soup.select_one('div.media_end_head_top a img')
        source = source_img_tag['alt'] if source_img_tag else "No Source Found"

        Nnews_tag = crawling_soup.select_one('#dic_area')
        Nnews = Nnews_tag.text.replace('\n', '').replace('\r', '').replace('\t', '') if Nnews_tag else "No Content Found"
        Writer_tag = crawling_soup.select_one('span.byline_s')
        Writer = Writer_tag.text if Writer_tag else "No Writer Found"

        writer.writerow([formatted_date, modify_date, title, Nnews, source, crawling_url, Writer])

def get_articles_for_date(date, writer):
    current_url = f"{base_url}{date.strftime('%Y%m%d')}"
    response = requests.get(current_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    iteration = 0
    extract_and_save_articles(soup, date, writer)

    while True:

        next_meta_div = soup.select_one('div.section_latest_article_CONTENT_LIST_PERSIST_META')
        
        if next_meta_div:
            next_cursor = next_meta_div.get('data-cursor', None)
        else:
            #print(f"No more articles to load for {date.strftime('%Y-%m-%d')}.")
            break  

        if next_cursor:

            next_url = f"{current_url}&data-cursor={next_cursor}"

            response = requests.get(next_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            iteration += 1
            extract_and_save_articles(soup, date, writer)

            time.sleep(1)
        else:
            print(f"No more data-cursor found for {date.strftime('%Y-%m-%d')}.")
            break  

start_date = datetime.strptime("20231212", "%Y%m%d")
end_date = datetime.strptime("20240830", "%Y%m%d")

current_date = start_date

file_exists = os.path.isfile(output_file)

with open(output_file, 'a' if file_exists else 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    if not file_exists:
        writer.writerow(['publication_date', 'update_date', 'title', 'content', 'publisher_id', 'article_url', 'author'])
    
    date_range = (end_date - start_date).days + 1
    for current_date in tqdm([start_date + timedelta(days=i) for i in range(date_range)], desc="Crawling Articles"):
        get_articles_for_date(current_date, writer)

print("Crawling End")


Crawling Articles:   1%|          | 2/263 [01:06<2:25:09, 33.37s/it]